In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query

from datetime import datetime

종목코드열람

In [2]:
shcode_list = query.query_shcode_list_today()

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [8]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 500
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 5:
            average_volume_test[ind] = 1
            
    return average_volume_test

In [6]:
today = datetime.now().strftime("%Y%m%d")

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to = today)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

In [9]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.05
volume_money = 20000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to = today)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v3' +'.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-11-25_10:51:32 : , 0 , 000020 , 동화약품 -------------------  
O_1. ratio_test 통과 0 , 000020 , 동화약품
X_2. total_volume_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-11-25_10:51:33 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-11-25_10:51:33 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-11-25_10:51:34 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-11-25_10:51:34 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-11-25_10:51:35 : , 5 , 000075 , 삼양홀딩스우 -------------------  
O_1. ratio_test 통과 5 , 000075 , 삼양홀딩스우
X_2. total_volume_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-11-25_10:51:35 : , 6 , 000080 , 하이트진로 -------------------  
O_1. ratio_test 통과 6 , 000080 , 하이트진로

X_1. ratio_test 없음 41 , 000650 , 천일고속
------------------- test 시작_2019-11-25_10:51:57 : , 42 , 000660 , SK하이닉스 -------------------  
O_1. ratio_test 통과 42 , 000660 , SK하이닉스
O_2. total_volume_test 통과 42 , 000660 , SK하이닉스
X_3. average_volume_test 없음 42 , 000660 , SK하이닉스
------------------- test 시작_2019-11-25_10:51:57 : , 43 , 000670 , 영풍 -------------------  
O_1. ratio_test 통과 43 , 000670 , 영풍
O_2. total_volume_test 통과 43 , 000670 , 영풍
X_3. average_volume_test 없음 43 , 000670 , 영풍
------------------- test 시작_2019-11-25_10:51:58 : , 44 , 000680 , LS네트웍스 -------------------  
O_1. ratio_test 통과 44 , 000680 , LS네트웍스
X_2. total_volume_test 없음 44 , 000680 , LS네트웍스
------------------- test 시작_2019-11-25_10:51:59 : , 45 , 000700 , 유수홀딩스 -------------------  
O_1. ratio_test 통과 45 , 000700 , 유수홀딩스
X_2. total_volume_test 없음 45 , 000700 , 유수홀딩스
------------------- test 시작_2019-11-25_10:51:59 : , 46 , 000720 , 현대건설 -------------------  
O_1. ratio_test 통과 46 , 000720 , 현대건설
O_2. total_volume_test 통

O_1. ratio_test 통과 83 , 001360 , 삼성제약
O_2. total_volume_test 통과 83 , 001360 , 삼성제약
O_3. average_volume_test 통과 83 , 001360 , 삼성제약
------------------- test 완료 : , 83 , 001360 , 삼성제약 ------------------- 
------------------- test 시작_2019-11-25_10:52:21 : , 84 , 001380 , SG충방 -------------------  
O_1. ratio_test 통과 84 , 001380 , SG충방
O_2. total_volume_test 통과 84 , 001380 , SG충방
O_3. average_volume_test 통과 84 , 001380 , SG충방
------------------- test 완료 : , 84 , 001380 , SG충방 ------------------- 
------------------- test 시작_2019-11-25_10:52:22 : , 85 , 001390 , KG케미칼 -------------------  
O_1. ratio_test 통과 85 , 001390 , KG케미칼
X_2. total_volume_test 없음 85 , 001390 , KG케미칼
------------------- test 시작_2019-11-25_10:52:23 : , 86 , 001420 , 태원물산 -------------------  
O_1. ratio_test 통과 86 , 001420 , 태원물산
X_2. total_volume_test 없음 86 , 001420 , 태원물산
------------------- test 시작_2019-11-25_10:52:23 : , 87 , 001430 , 세아베스틸 -------------------  
O_1. ratio_test 통과 87 , 001430 , 세아베스틸
X_2. total_volu

O_1. ratio_test 통과 124 , 002025 , 코오롱우
X_2. total_volume_test 없음 124 , 002025 , 코오롱우
------------------- test 시작_2019-11-25_10:52:45 : , 125 , 002030 , 아세아 -------------------  
X_1. ratio_test 없음 125 , 002030 , 아세아
------------------- test 시작_2019-11-25_10:52:46 : , 126 , 002070 , 남영비비안 -------------------  
O_1. ratio_test 통과 126 , 002070 , 남영비비안
O_2. total_volume_test 통과 126 , 002070 , 남영비비안
O_3. average_volume_test 통과 126 , 002070 , 남영비비안
------------------- test 완료 : , 126 , 002070 , 남영비비안 ------------------- 
------------------- test 시작_2019-11-25_10:52:46 : , 127 , 002100 , 경농 -------------------  
O_1. ratio_test 통과 127 , 002100 , 경농
O_2. total_volume_test 통과 127 , 002100 , 경농
O_3. average_volume_test 통과 127 , 002100 , 경농
------------------- test 완료 : , 127 , 002100 , 경농 ------------------- 
------------------- test 시작_2019-11-25_10:52:47 : , 128 , 002140 , 고려산업 -------------------  
O_1. ratio_test 통과 128 , 002140 , 고려산업
O_2. total_volume_test 통과 128 , 002140 , 고려산업
O_3. avera

O_1. ratio_test 통과 161 , 002810 , 삼영무역
X_2. total_volume_test 없음 161 , 002810 , 삼영무역
------------------- test 시작_2019-11-25_10:53:7 : , 162 , 002820 , 선창산업 -------------------  
X_1. ratio_test 없음 162 , 002820 , 선창산업
------------------- test 시작_2019-11-25_10:53:7 : , 163 , 002840 , 미원상사 -------------------  
O_1. ratio_test 통과 163 , 002840 , 미원상사
X_2. total_volume_test 없음 163 , 002840 , 미원상사
------------------- test 시작_2019-11-25_10:53:8 : , 164 , 002870 , 신풍제지 -------------------  
O_1. ratio_test 통과 164 , 002870 , 신풍제지
X_2. total_volume_test 없음 164 , 002870 , 신풍제지
------------------- test 시작_2019-11-25_10:53:9 : , 165 , 002880 , 대유에이텍 -------------------  
O_1. ratio_test 통과 165 , 002880 , 대유에이텍
O_2. total_volume_test 통과 165 , 002880 , 대유에이텍
X_3. average_volume_test 없음 165 , 002880 , 대유에이텍
------------------- test 시작_2019-11-25_10:53:9 : , 166 , 002900 , 동양물산 -------------------  
O_1. ratio_test 통과 166 , 002900 , 동양물산
O_2. total_volume_test 통과 166 , 002900 , 동양물산
O_3. average_volume

...

O_1. ratio_test 통과 2775 , 250000 , 보라티알
O_2. total_volume_test 통과 2775 , 250000 , 보라티알
O_3. average_volume_test 통과 2775 , 250000 , 보라티알
------------------- test 완료 : , 2775 , 250000 , 보라티알 ------------------- 
------------------- test 시작_2019-11-25_11:20:31 : , 2776 , 250060 , 모비스 -------------------  
O_1. ratio_test 통과 2776 , 250060 , 모비스
O_2. total_volume_test 통과 2776 , 250060 , 모비스
O_3. average_volume_test 통과 2776 , 250060 , 모비스
------------------- test 완료 : , 2776 , 250060 , 모비스 ------------------- 
------------------- test 시작_2019-11-25_11:20:32 : , 2777 , 250930 , 예선테크 -------------------  
O_1. ratio_test 통과 2777 , 250930 , 예선테크
O_2. total_volume_test 통과 2777 , 250930 , 예선테크
O_3. average_volume_test 통과 2777 , 250930 , 예선테크
------------------- test 완료 : , 2777 , 250930 , 예선테크 ------------------- 
------------------- test 시작_2019-11-25_11:20:33 : , 2778 , 251370 , 와이엠티 -------------------  
O_1. ratio_test 통과 2778 , 251370 , 와이엠티
X_2. total_volume_test 없음 2778 , 251370 , 와이엠티
---

O_1. ratio_test 통과 2808 , 263770 , 유에스티
X_2. total_volume_test 없음 2808 , 263770 , 유에스티
------------------- test 시작_2019-11-25_11:20:55 : , 2809 , 263800 , 데이타솔루션 -------------------  
O_1. ratio_test 통과 2809 , 263800 , 데이타솔루션
O_2. total_volume_test 통과 2809 , 263800 , 데이타솔루션
O_3. average_volume_test 통과 2809 , 263800 , 데이타솔루션
------------------- test 완료 : , 2809 , 263800 , 데이타솔루션 ------------------- 
------------------- test 시작_2019-11-25_11:20:56 : , 2810 , 263810 , 상신전자 -------------------  
O_1. ratio_test 통과 2810 , 263810 , 상신전자
X_2. total_volume_test 없음 2810 , 263810 , 상신전자
------------------- test 시작_2019-11-25_11:20:56 : , 2811 , 263860 , 지니언스 -------------------  
O_1. ratio_test 통과 2811 , 263860 , 지니언스
X_2. total_volume_test 없음 2811 , 263860 , 지니언스
------------------- test 시작_2019-11-25_11:20:57 : , 2812 , 263920 , 블러썸엠앤씨 -------------------  
O_1. ratio_test 통과 2812 , 263920 , 블러썸엠앤씨
X_2. total_volume_test 없음 2812 , 263920 , 블러썸엠앤씨
------------------- test 시작_2019-11-25_11:20:5

O_1. ratio_test 통과 2845 , 286750 , 나노브릭
O_2. total_volume_test 통과 2845 , 286750 , 나노브릭
X_3. average_volume_test 없음 2845 , 286750 , 나노브릭
------------------- test 시작_2019-11-25_11:21:20 : , 2846 , 287410 , 유안타제3호스팩 -------------------  
X_1. ratio_test 없음 2846 , 287410 , 유안타제3호스팩
------------------- test 시작_2019-11-25_11:21:21 : , 2847 , 288620 , 에스퓨얼셀 -------------------  
O_1. ratio_test 통과 2847 , 288620 , 에스퓨얼셀
O_2. total_volume_test 통과 2847 , 288620 , 에스퓨얼셀
O_3. average_volume_test 통과 2847 , 288620 , 에스퓨얼셀
------------------- test 완료 : , 2847 , 288620 , 에스퓨얼셀 ------------------- 
------------------- test 시작_2019-11-25_11:21:22 : , 2848 , 289010 , 아이스크림에듀 -------------------  
O_1. ratio_test 통과 2848 , 289010 , 아이스크림에듀
O_2. total_volume_test 통과 2848 , 289010 , 아이스크림에듀
X_3. average_volume_test 없음 2848 , 289010 , 아이스크림에듀
------------------- test 시작_2019-11-25_11:21:22 : , 2849 , 289080 , SV인베스트먼트 -------------------  
O_1. ratio_test 통과 2849 , 289080 , SV인베스트먼트
O_2. total_volume_test 통과

X_1. ratio_test 없음 2880 , 307280 , 교보8호스팩
------------------- test 시작_2019-11-25_11:21:43 : , 2881 , 307750 , 대신밸런스제6호스팩 -------------------  
X_1. ratio_test 없음 2881 , 307750 , 대신밸런스제6호스팩
------------------- test 시작_2019-11-25_11:21:44 : , 2882 , 307870 , 상상인이안1호스팩 -------------------  
X_1. ratio_test 없음 2882 , 307870 , 상상인이안1호스팩
------------------- test 시작_2019-11-25_11:21:45 : , 2883 , 307930 , 컴퍼니케이 -------------------  
O_1. ratio_test 통과 2883 , 307930 , 컴퍼니케이
O_2. total_volume_test 통과 2883 , 307930 , 컴퍼니케이
O_3. average_volume_test 통과 2883 , 307930 , 컴퍼니케이
------------------- test 완료 : , 2883 , 307930 , 컴퍼니케이 ------------------- 
------------------- test 시작_2019-11-25_11:21:45 : , 2884 , 308100 , 까스텔바작 -------------------  
O_1. ratio_test 통과 2884 , 308100 , 까스텔바작
O_2. total_volume_test 통과 2884 , 308100 , 까스텔바작
O_3. average_volume_test 통과 2884 , 308100 , 까스텔바작
------------------- test 완료 : , 2884 , 308100 , 까스텔바작 ------------------- 
------------------- test 시작_2019-11-25_11:21:4

X_1. ratio_test 없음 2920 , 331380 , 유진스팩5호
------------------- test 시작_2019-11-25_11:22:10 : , 2921 , 331520 , 교보9호스팩 -------------------  
X_1. ratio_test 없음 2921 , 331520 , 교보9호스팩
------------------- test 시작_2019-11-25_11:22:11 : , 2922 , 332290 , 대신밸런스제7호스팩 -------------------  
X_1. ratio_test 없음 2922 , 332290 , 대신밸런스제7호스팩
------------------- test 시작_2019-11-25_11:22:11 : , 2923 , 332710 , 하나금융14호스팩 -------------------  
X_1. ratio_test 없음 2923 , 332710 , 하나금융14호스팩
------------------- test 시작_2019-11-25_11:22:12 : , 2924 , 333050 , 신한제6호스팩 -------------------  
X_1. ratio_test 없음 2924 , 333050 , 신한제6호스팩
------------------- test 시작_2019-11-25_11:22:13 : , 2925 , 333430 , 미래에셋대우스팩4호 -------------------  
X_1. ratio_test 없음 2925 , 333430 , 미래에셋대우스팩4호
------------------- test 시작_2019-11-25_11:22:13 : , 2926 , 900040 , 차이나그레이트 -------------------  
O_1. ratio_test 통과 2926 , 900040 , 차이나그레이트
O_2. total_volume_test 통과 2926 , 900040 , 차이나그레이트
O_3. average_volume_test 통과 2926 , 900040 , 차이나그